In [1]:
%matplotlib inline
from __future__ import division
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import math
from adjustText import adjust_text

import common
import notebook_helpers

FS = (10, 7)

om = notebook_helpers.get_frame()
lyrs = lyr = notebook_helpers.get_frame(having_lyrics=True)

In [70]:
import json
def to_js(df, fname, varname="DATA", pprint=1):
    json_str = df.to_json(None, orient='records')
    if pprint:
        json_str = json_str.replace('},', '},\n  ')
    _json_str_to_js(json_str, fname, varname)
    
def dict_to_js(d, fname, varname="DATA", pprint=1):
    s = json.dumps(d, indent=(2 if pprint else None))
    _json_str_to_js(s, fname, varname)
    
def _json_str_to_js(json_str, fname, varname):
    if not fname.endswith('.js'):
        fname += '.js'
    with open(fname, 'w') as f:
        f.write('var {} = '.format(varname))
        f.write(json_str)
        f.write(';\nexport default {};'.format(varname))
    print "Wrote results to {}".format(fname)
    os.system('cp {} ~/src/lalala/src/js/'.format(fname))

In [77]:
# Data for the repetition over the years chart
from scipy.stats import hmean
bookends = 2 # how many years to take off front and back
window_size = 3
rolling = 1
# Harmonic mean made sense when using ratios, but since we're now using 'rscores', which are
# in log space, we can just use the arithmetic mean.
use_harmonic_mean = 0
ihit = lyrs['peak'] <= 10
def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

if use_harmonic_mean:
    mean_per_year = lyrs.groupby('year')['rscore'].apply(hmean)
else:
    mean_per_year = lyrs.groupby('year')['rscore'].mean()
if rolling:
    y = movingaverage(mean_per_year.values, window_size)
else:
    y = mean_per_year.values
yrs = mean_per_year.index.values
frame = np.stack([yrs, y]).T
ot = pd.DataFrame(frame, columns=['year', 'rscore'])


if use_harmonic_mean:
    mean_per_year = lyrs[ihit].groupby('year')['rscore'].apply(hmean)
else:
    mean_per_year = lyrs[ihit].groupby('year')['rscore'].mean()
mean_per_year = mean_per_year.loc[yrs]
if rolling:
    y = movingaverage(mean_per_year.values, window_size)
else:
    y = mean_per_year.values
ot['hitsRscore'] = y
ot  = ot.set_index('year')


# Get each year's most repetitive song. I hate pandas.
idm = lyrs.groupby('year')['rscore'].idxmax()
mostrep = lyrs.ix[idm,['year', 'artist', 'title', 'rscore']]
mos = mostrep.set_index('year')
dat = []

for y in yrs[bookends:-bookends]:
    row = ot.loc[y].to_dict()
    row['year'] = y
    row['topsong'] = mos.loc[y].to_dict()
    dat.append(row)

dict_to_js(dat, 'years.js', varname='DATA')

Wrote results to years.js


In [78]:
ot.tail()

,rscore,hitsRscore
year,,
2013.0,1.144306,1.259210
2014.0,1.153249,1.269777
2015.0,1.151047,1.264940
2016.0,1.139138,1.292910
2017.0,0.754364,0.862035


In [45]:
# Data for the artist repetitiveness chart
min_songs = 20

songs_per_artist = lyrs.groupby('artist').size()
stars = songs_per_artist[songs_per_artist >= min_songs].to_frame(name='nsongs')
istar = stars.index

stars['year'] = lyrs.groupby('artist')['year'].median().loc[istar].astype(int)
stars['rscore'] = lyrs.groupby('artist')['rscore'].median().loc[istar]
stars['name'] = stars.index

# Get each artist's most repetitive song. I hate pandas.
idm = lyrs.groupby('artist')['rscore'].idxmax()
mostrep = lyrs.ix[idm,['artist', 'title', 'rscore']]
mos = pd.Series(data=mostrep['title'].values, index=mostrep['artist'].values)
moss = pd.Series(data=mostrep['rscore'].values, index=mostrep['artist'].values)
stars['mostrep'] = mos[istar]
stars['topscore'] = moss[istar]

print "Got {} stars".format(len(stars))
pprint = 1
json_str = stars.to_json(None, orient='records', lines=0)
if pprint:
    json_str = json_str.replace('},', '},\n  ')
with open('stars.json', 'w') as f:
    f.write('var artists = ')
    f.write(json_str)
    f.write(';\nexport default artists;')
os.system('cp stars.json ~/src/lalala/src/js/artist-data.js')
stars.head()

Got 109 stars


,nsongs,year,rscore,name,mostrep,topscore
artist,,,,,,
ABBA,20,1977,1.103363,ABBA,Voulez-vous,1.529247
Aerosmith,29,1989,1.045021,Aerosmith,Rag Doll,1.456969
Alan Jackson,21,2002,1.015606,Alan Jackson,It Must Be Love,1.548216
Andy Williams,26,1964,0.832609,Andy Williams,"Promise Me, Love",1.448871
Aretha Franklin,48,1971,1.036882,Aretha Franklin,United Together,1.452170


In [40]:
type(json_str)

str

In [35]:
mostrep = lyrs.ix[idm,['artist', 'title']]
mos = pd.Series(data=mostrep['title'].values, index=mostrep['artist'].values)

'Til Tuesday                    Coming Up Close
(+44)             When Your Heart Stops Beating
10 Years                              Wasteland
10,000 Maniacs            Candy Everybody Wants
10cc                         Good Morning Judge
dtype: object